In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
data=pd.read_csv("D:/programming-no-gdrive/air-pollution/data-formatted/03.03.19-restrict-hours/all_sites-2000.csv")

This data has no nulls in it. Currently testing on just 2000 data. My idea is that we should do one interpolation with 2000 and one with 2017. 

In [5]:
data['month'].value_counts()

12.0    19144
10.0    18933
8.0     18694
11.0    18097
9.0     17813
7.0     17096
3.0     16353
5.0     16257
4.0     15996
6.0     15832
2.0     14833
Name: month, dtype: int64

Last time I made a note that January had far less data than other months so I would drop it. Here it is already not represented at all.

In [14]:
X_train, X_test, y_train, y_test=train_test_split(data.drop(['o3'],axis = 1) , data[['o3']])

<h2>First Linear Regression</h2>

In [15]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(X_train,y_train)
print(reg.score(X_train,y_train))

0.03847215198119269


<h2>Random Forest</h2>

First try without normalization

In [16]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 10, random_state = 0, verbose = 1)
# Train the model on training data
rf.fit(X_train, y_train)

C:\Users\nicholas\Anaconda3\envs\deep-learning\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   12.2s finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=0, verbose=1, warm_start=False)

In [18]:
print(rf.score(X_test, y_test))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.42225545633116046


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished


In [21]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf1000 = RandomForestRegressor(n_estimators = 1000, random_state = 0, verbose = 1)
# Train the model on training data
rf1000.fit(X_train, y_train)

C:\Users\nicholas\Anaconda3\envs\deep-learning\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed: 20.0min finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
           oob_score=False, random_state=0, verbose=1, warm_start=False)

In [22]:
print(rf1000.score(X_test, y_test))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.47667216690489556


[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:   20.9s finished


In [23]:
from sklearn.externals import joblib
joblib_file = "hrly_rf_10.pkl"  
joblib.dump(rf, joblib_file)

['hrly_rf_10.pkl']